# Introdução

Esta EDA analisa os relatórios de incidentes criminais na cidade de Boston, de junho de 2015 a setembro de 2018. Uso o Folium para traçar um mapa interativo de calor de Boston e para o mar para todo o resto.

Os dados são originalmente fornecidos pelo hub de dados abertos de Boston, [Analyze Boston] (https://data.boston.gov/dataset/crime-incident-reports-august-2015-to-date-source-new-system). Este [kernel] (https://www.kaggle.com/kosovanolexandr/crimes-in-boston-multiclass-clustering) de [Kosovan Olexandr] (https://www.kaggle.com/kosovanolexandr) me ajudou a começar este conjunto de dados e este [outro kernel] (https://www.kaggle.com/daveianhickey/how-to-folium-for-maps-heatmaps-time-analysis) por [Dave Fisher-Hickey] (https: // www.kaggle.com/daveianhickey) me ajudou a começar a usar o Folium.

In [ ]:
# Carregando bibliotecas
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline 
import seaborn as sns
import folium
from folium.plugins import HeatMap

# Importando dados
data = pd.read_csv('../input/crime.csv', encoding='latin-1')

# Olhadinha
data.head()

Primeiro, vamos limpar e simplificar esse conjunto de dados. Vou focar nos dois anos com dados completos (2016 e 2017). Também abordarei as ofensas [UCR Parte Um] (https://www.ucrdatatool.gov/offenses.cfm) que incluem apenas os crimes mais graves.

In [ ]:
# somente dados de anos completos (2016, 2017)
data = data.loc[data['YEAR'].isin([2016,2017])]
# Manter apenas dados sobre ofensas da Parte 1 do UCR
data = data.loc[data['UCR_PART'] == 'Part One']

# Remover colunas não utilizadas
data = data.drop(['INCIDENT_NUMBER','OFFENSE_CODE','UCR_PART','Location'], axis=1)

# Converter OCCURED_ON_DATE em data e hora
data['OCCURRED_ON_DATE'] = pd.to_datetime(data['OCCURRED_ON_DATE'])

# Preencha nans na coluna SHOOTING
data.SHOOTING.fillna('N', inplace=True)

# Converter DAY_OF_WEEK em uma categoria ordenada
data.DAY_OF_WEEK = pd.Categorical(data.DAY_OF_WEEK, 
              categories=['Monday','Tuesday','Wednesday','Thursday','Friday','Saturday','Sunday'],
              ordered=True)

# Substitua -1 valores em Lat / Long por Nan
data.Lat.replace(-1, None, inplace=True)
data.Long.replace(-1, None, inplace=True)

# Renomeie as colunas para algo mais fácil de digitar (as letras maiúsculas são irritantes!)
rename = {'OFFENSE_CODE_GROUP':'Group',
         'OFFENSE_DESCRIPTION':'Description',
         'DISTRICT':'District',
         'REPORTING_AREA':'Area',
         'SHOOTING':'Shooting',
         'OCCURRED_ON_DATE':'Date',
         'YEAR':'Year',
         'MONTH':'Month',
         'DAY_OF_WEEK':'Day',
         'HOUR':'Hour',
         'STREET':'Street'}
data.rename(index=str, columns=rename, inplace=True)

# Check
data.head()

In [ ]:
# Mais algumas verificações de dados
data.dtypes
data.isnull().sum()
data.shape


# Tipos de crimes graves

Vamos começar verificando a frequência de diferentes tipos de crimes. Uma vez que atribuímos apenas crimes 'sérios', existem apenas 9 tipos diferentes de crimes - muito mais administráveis do que os 67 que começamos.

In [ ]:
# Gráfico de contagem para tipos de crime
sns.catplot(y='Group',
           kind='count',
            height=8, 
            aspect=1.5,
            order=data.Group.value_counts().index,
           data=data)

O furto é de longe o crime mais comum e os homicídios são bastante raros.

# Quando ocorrem crimes graves?

Podemos considerar padrões em várias escalas de tempo diferentes: horas do dia, dias da semana e meses do ano.

In [ ]:
# Crimes por hora do dia
sns.catplot(x='Hour',
           kind='count',
            height=8.27, 
            aspect=3,
            color='black',
           data=data)
plt.xticks(size=30)
plt.yticks(size=30)
plt.xlabel('Hour', fontsize=40)
plt.ylabel('Count', fontsize=40)

In [ ]:
# Crimes por dia da semana
sns.catplot(x='Day',
           kind='count',
            height=8, 
            aspect=3,
           data=data)
plt.xticks(size=30)
plt.yticks(size=30)
plt.xlabel('')
plt.ylabel('Count', fontsize=40)

In [ ]:
# Crimes por mês do ano
months = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
sns.catplot(x='Month',
           kind='count',
            height=8, 
            aspect=3,
            color='gray',
           data=data)
plt.xticks(np.arange(12), months, size=30)
plt.yticks(size=30)
plt.xlabel('')
plt.ylabel('Count', fontsize=40)

As taxas de crimes são baixas entre 1 e 8 da manhã e aumentam gradualmente ao longo do dia, chegando às 18h. Há algumas variações entre os dias da semana, com sexta-feira com a maior taxa de criminalidade e domingo com a menor. O mês também parece ter alguma influência, com os meses de inverno de fevereiro a abril com as menores taxas de criminalidade e os meses de verão / outono de junho a outubro com as maiores taxas de criminalidade. Há também um aumento nas taxas de criminalidade no mês de janeiro.

Existem outros fatores temporais associados ao crime? [De acordo com alguns especialistas em crime] (https://www.oxygen.com/homicide-for-the-holidays/blogs/its-the-most-dangerous-time-of-the-year-why-do-crimes- aumento), vários tipos de crimes tendem a aumentar em torno dos feriados, principalmente larsony e roubo. Isso pode ocorrer por várias razões: shopping centers lotados criam mais cobertura para ladrões, os viajantes deixam suas casas vulneráveis a roubos e o aumento do uso de álcool e drogas pode aumentar a probabilidade de crimes relacionados a conflitos. Vamos ver se há alguma evidência disso em nossos dados, com foco no ano de 2017. Também adicionei em alguns dias que são conhecidos por serem especialmente turbulentos em Boston, mesmo que não sejam feriados oficiais: St. Patrick's Dia e Maratona de Boston.

In [ ]:
# Criar dados para plotagem
data['Day_of_year'] = data.Date.dt.dayofyear
data_holidays = data[data.Year == 2017].groupby(['Day_of_year']).size().reset_index(name='counts')

# Datas dos principais feriados nos EUA em 2017
holidays = pd.Series(['2017-01-01', # New Years Day
                     '2017-01-16', # MLK Day
                     '2017-03-17', # St. Patrick's Day
                     '2017-04-17', # Boston marathon
                     '2017-05-29', # Memorial Day
                     '2017-07-04', # Independence Day
                     '2017-09-04', # Labor Day
                     '2017-10-10', # Veterans Day
                     '2017-11-23', # Thanksgiving
                     '2017-12-25']) # Christmas
holidays = pd.to_datetime(holidays).dt.dayofyear
holidays_names = ['NY',
                 'MLK',
                 'St Pats',
                 'Marathon',
                 'Mem',
                 'July 4',
                 'Labor',
                 'Vets',
                 'Thnx',
                 'Xmas']

import datetime as dt
# Crimes de conspiração e feriados
fig, ax = plt.subplots(figsize=(11,6))
sns.lineplot(x='Day_of_year',
            y='counts',
            ax=ax,
            data=data_holidays)
plt.xlabel('Day of the year')
plt.vlines(holidays, 20, 80, alpha=0.5, color ='r')
for i in range(len(holidays)):
    plt.text(x=holidays[i], y=82, s=holidays_names[i])

Hum, não estou vendo nenhum sinal claro aqui. De fato, muitos desses feriados parecem se alinhar com taxas de criminalidade especialmente baixas, particularmente no Dia de Ação de Graças e no Natal. Obviamente, trata-se de dados de apenas um ano, e detectar uma associação entre um determinado feriado e as taxas de criminalidade exigiria muito mais dados e um modelo que explique outros fatores. No entanto, isso me leva a questionar a idéia geral de que o crime aumenta nos feriados - se isso é verdade, não é super óbvio do ponto de vista dos dados. Até toda a ["temporada de festas"] (https://www.cpss.net/about/blog/2013/11/stay-safe-crime-rates-increase-during-holiday-season/) do Dia de Ação de Graças ao Natal parece estar especialmente elevado em comparação com o verão.

# Onde ocorrem crimes graves?

Podemos usar as colunas de latitude e longitude para traçar a localização dos crimes em Boston. Ao definir o parâmetro alfa para um valor muito pequeno, podemos ver que existem alguns 'pontos quentes' do crime.

In [ ]:
#Gráfico de dispersão simples
sns.scatterplot(x='Lat',
               y='Long',
                alpha=0.01,
               data=data)

Parece Boston bem. Se você conhece Boston, não ficará surpreso ao ver que o centro de Boston tem os pontos mais sombrios, mas também existem algumas localidades fora do centro da cidade que apresentam índices de criminalidade especialmente altos. 

Vamos fazer outro gráfico de dispersão, mas desta vez vamos colorir pontos por distrito para ver quais distritos têm as maiores taxas de criminalidade.

In [ ]:
# Distritos da trama
sns.scatterplot(x='Lat',
               y='Long',
                hue='District',
                alpha=0.01,
               data=data)
plt.legend(bbox_to_anchor=(1.05, 1), loc=2)

Agora podemos associar altas taxas de criminalidade a distritos específicos, notadamente A1 e D4, que correspondem às áreas mais movimentadas do centro de Boston.

Há também uma visibilidade muito alta da região criminosa no distrito D14. Vamos deixar as coisas bonitas usando o Folium para criar um mapa interativo dos crimes de Boston. Usarei os dados de 2017 apenas para esse gráfico.

In [ ]:
#Criar mapa básico do crime Folium
crime_map = folium.Map(location=[42.3125,-71.0875], 
                       tiles = "Stamen Toner",
                      zoom_start = 11)

# Adicionar dados para o mapa de calor
data_heatmap = data[data.Year == 2017]
data_heatmap = data[['Lat','Long']]
data_heatmap = data.dropna(axis=0, subset=['Lat','Long'])
data_heatmap = [[row['Lat'],row['Long']] for index, row in data_heatmap.iterrows()]
HeatMap(data_heatmap, radius=10).add_to(crime_map)

# Enredo!
crime_map

# Conclusão

Em resumo, esta EDA mostra:

* O furto é de longe o tipo mais comum de crime grave. 
* É provável que crimes sérios ocorram à tarde e à noite. 
* É provável que crimes graves ocorram na sexta-feira e menos no domingo. 
* É provável que crimes graves ocorram no verão e no início do outono e menos no inverno (com exceção de janeiro, que tem uma taxa de criminalidade mais semelhante ao verão). 
* Não há conexão óbvia entre os principais feriados e as taxas de criminalidade. 
* Crimes graves são mais comuns no centro da cidade, especialmente nos distritos A1 e D4.

Este EDA apenas arranha a superfície do conjunto de dados. Análises posteriores poderiam explorar como os diferentes tipos de crimes variam no tempo e no espaço. Eu nem sequer considerei os crimes menos graves da parte dois e da parte três da UCR, que são muito mais comuns que os crimes da parte um, mas incluem categorias interessantes, como crimes por drogas. Outra direção interessante seria combinar isso com outros dados sobre Boston, como demografia ou até o clima (http://www.chicagotribune.com/news/data/ct-crime-heat-analysis-htmlstory.html) , para investigar quais fatores preveem as taxas de criminalidade no tempo e no espaço.